In [22]:

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
from pyspark.sql import functions as f
from pyspark.sql.types import *
import datetime
import dateutil.relativedelta
import os
import sys
import argparse

spark = SparkSession.builder \
.master("local") \
.appName("CS in HT") \
.getOrCreate()

In [23]:
unp = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Unpaid_Services_-_DocNum_sample.csv")
unp.printSchema()
unp.registerTempTable('u')
unp.show(2)

root
 |-- Keydate: integer (nullable = true)
 |-- IS_Locked: integer (nullable = true)
 |-- OIB: string (nullable = true)
 |-- CONTRACT_ACCOUNT: long (nullable = true)
 |-- Document_Type: string (nullable = true)
 |-- Document_Number: long (nullable = true)
 |-- Amount: double (nullable = true)

+--------+---------+-----------+----------------+-------------+---------------+------+
| Keydate|IS_Locked|        OIB|CONTRACT_ACCOUNT|Document_Type|Document_Number|Amount|
+--------+---------+-----------+----------------+-------------+---------------+------+
|20170731|        1|63163103332|    200000460514|           CF|      630744178|  13.9|
|20170731|        1|47005345002|    200002209040|           CF|      631583848| 312.5|
+--------+---------+-----------+----------------+-------------+---------------+------+
only showing top 2 rows



In [24]:
collvl = spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/CollectionLevel_by_DocNo_sample.csv")
collvl.printSchema()

collvl.registerTempTable('c')


root
 |-- Document_Number: long (nullable = true)
 |-- Due_Date: integer (nullable = true)
 |-- Dunning_Date: integer (nullable = true)
 |-- Collection_Level: integer (nullable = true)
 |-- Dunning_Amount: double (nullable = true)



In [25]:
unp_df = spark.sql("select * from u where Document_Type not in ('CF','EN','IC','NP','PO','SR')")

In [26]:
lock=spark.read.option("header",True).option("inferSchema",True).option("delimiter",";")\
   .csv("file:////home/abhijeet/spark coding/Data_sample/Lock_by_DocNum_-_Complaints.csv")
lock.printSchema()

df=unp_df.join(lock,"document_number","inner")
print(df)
df.registerTempTable('l')

root
 |-- Document_Number: long (nullable = true)
 |-- Lock_From_Date: integer (nullable = true)
 |-- Lock_To_Date: integer (nullable = true)

DataFrame[Document_Number: bigint, Keydate: int, IS_Locked: int, OIB: string, CONTRACT_ACCOUNT: bigint, Document_Type: string, Amount: double, Lock_From_Date: int, Lock_To_Date: int]


In [27]:
df1=spark.sql("SELECT * FROM l WHERE l.Keydate not between l.Lock_From_Date AND l.Lock_To_Date").show()

+---------------+--------+---------+-----------+----------------+-------------+--------+--------------+------------+
|Document_Number| Keydate|IS_Locked|        OIB|CONTRACT_ACCOUNT|Document_Type|  Amount|Lock_From_Date|Lock_To_Date|
+---------------+--------+---------+-----------+----------------+-------------+--------+--------------+------------+
|    20129060819|20170731|        0|58681530186|      5007023506|           RK| 2997.96|      20170525|    20170606|
|    20301893585|20170731|        0| 4980538323|      5006918528|           RK|  249.63|      20170803|    20171101|
|    19222991672|20170731|        0| 6709427961|    200003423380|           IN|   139.0|      20171019|    20171218|
|    19203031350|20170731|        0|92566034883|    200002663922|           IN|  1893.7|      20171009|    20171208|
|    20221831970|20170731|        0| 7323353416|      5006392656|           RK|  846.51|      20170110|    20170410|
|    20271704395|20170731|        0|72252041403|      5030045850

In [ ]:
join = df.join(collvl,"document_number","inner")
print(join)
join.registerTempTable('j')

In [ ]:
df1=spark.sql("select Document_Number,amount,oib,collection_level,Dunning_Date,Dunning_Amount from j \
where Dunning_Date<=Keydate")
df1.show()
df1.count()

In [ ]:
SELECT u.document_number,u.keydate_prompt,u.amount,u.oib,c.collection_level,
c.dunning_date,c.dunning_amount
FROM stg_dwh.stg_sas_collectionlevel_by_docno c inner join
(select * from stg_dwh.stg_sas_unpaid_services_by_docnum 
where Document_Type not in ('CF','EN','IC','NP','PO','SR') 
and is_locked_on_keydate='0') u  where c.due_date<=u.keydate_prompt;